## Analysis

In [1]:
import A1_provider_prep
import A2_other
import A3_analysis
import geopandas as gpd
import pandas as pd
from calitp.sql import to_snakecase
from shared_utils import geography_utils, utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
# Read in no coverage maps
verizon = A1_provider_prep.load_verizon()

In [4]:
att = A1_provider_prep.load_att()

In [5]:
tmobile = A1_provider_prep.load_tmobile()

In [6]:
# att.plot()

In [7]:
# Original AT&T map before finding difference
# gpd.read_parquet("./ATT.parquet").plot()

### Which routes touch areas without data coverage among AT&T, Verizon, and T-Mobile?

In [8]:
# Find routes that run in areas without coverage
# Across all 3 providers
final = A3_analysis.final_merge(A3_analysis.merge_all_providers())

In [9]:
# Load in original geometries of the routes.
one_dist_routes, multi_dist_routes, all_routes = A2_other.find_multi_district_routes()

In [12]:
f"There are {len(final)} routes that touch a zone w/o data coverage among the poviders out of {len(all_routes)} routes. There are {final.Agency.nunique()} agencies in this dataset."

'There are 2658 routes that touch a zone w/o data coverage among ATT, T-Mobile, and Verizon out of 2914 routes. There are 166 agencies in this dataset.'

In [11]:
low_coverage = A3_analysis.final_merge(routes1)

NameError: name 'routes1' is not defined

### Narrow down low coverage routes
* Most routes (776 out of 865) have data coverage for 90-100% of the routes.
* Threshold based on the median % of the route with data coverage across the 3 providers.

In [13]:
# Create bins to see the median percentage of the route WITH coverage.
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
final["Binned"] = pd.cut(final["Median Percent With Coverage"], bins)

In [14]:
final["Binned"].value_counts()

(90, 100]    2565
(80, 90]       42
(70, 80]       31
(60, 70]        9
(50, 60]        6
(40, 50]        3
(30, 40]        2
(0, 10]         0
(10, 20]        0
(20, 30]        0
Name: Binned, dtype: int64

In [15]:
final["Median Percent No Coverage"].describe()

count   2658.00
mean       1.03
std        4.97
min        0.00
25%        0.00
50%        0.00
75%        0.00
max       64.00
Name: Median Percent No Coverage, dtype: float64

In [16]:
final["Median Percent With Coverage"].describe()

count   2658.00
mean      98.97
std        4.97
min       36.00
25%      100.00
50%      100.00
75%      100.00
max      100.00
Name: Median Percent With Coverage, dtype: float64

In [60]:
threshold = 75

In [61]:
low_coverage = (
    final.loc[final["Median Percent With Coverage"] < threshold]
).reset_index(drop=True)

In [62]:
f"{len(low_coverage)} routes are considered low-data coverage"

'32 routes are considered low-data coverage'

In [63]:
subset = [
    "Agency",
    "Long Route Name",
    "District",
    "Median Percent With Coverage",
    "Median Percent No Coverage",
    "Total Trips By Route",
    "Total Buses",
    "Estimate Of Buses In Low Cell Zones",
]

### Conclusions

In [64]:
low_coverage[subset].sort_values("Median Percent No Coverage", ascending=False)

,Agency,Long Route Name,District,Median Percent With Coverage,Median Percent No Coverage,Total Trips By Route,Total Buses,Estimate Of Buses In Low Cell Zones
1,Trinity Transit,Route travels through all down-river communities between Weaverville and Willow Creek. Key route to the coast; connects with RTS service to Arcata/Eureka. 181 Trinity Transit,"D-1, D-2",36.00,64.00,4,7.00,1
24,Trinity Transit,Route between Weaverville and Hayfork. Connects to Redding Line in Douglas City. 178 Trinity Transit,D-2,39.00,61.00,4,7.00,1
19,Mendocino Transit Authority,The Coaster 230 Mendocino Transit Authority,D-1,45.00,55.00,4,35.00,1
26,Trinity Transit,Redding Line 288 Trinity Transit,D-2,46.00,54.00,4,7.00,1
31,Yosemite Area Regional Transportation System,Mammoth Lakes HWY 120E/395 582 Yosemite Area Regional Transportation System,"D-10, D-9",48.00,52.00,2,10.00,1
28,Tulare County Regional Transit Agency,Porterville to Springville Loop c70 Tulare County Regional Transit Agency,D-6,53.00,47.00,4,18.00,1
27,Tulare County Area Transit,Porterville to Springville Loop 12181 Tulare County Area Transit,D-6,53.00,47.00,4,24.00,1
21,Redwood Coast Transit,Route 199 Hiouchi / Gasquet 257 Redwood Coast Transit,D-1,55.00,45.00,6,14.00,1
29,Yosemite Area Regional Transportation System,Sonora HWY 120 1094 Yosemite Area Regional Transportation System,D-10,55.00,45.00,2,10.00,1
30,Yosemite Area Regional Transportation System,Merced HWY 140 580 Yosemite Area Regional Transportation System,D-10,59.00,41.00,17,10.00,8


In [65]:
f"Around {low_coverage['Estimate Of Buses In Low Cell Zones'].sum()} buses run through low data coverage routes."

'Around 85 buses run through low data coverage routes.'

In [66]:
f"There {low_coverage['Long Route Name'].nunique()} low data coverage routes."

'There 32 low data coverage routes.'

In [67]:
low_coverage.District.value_counts()

D-4          8
D-1          7
D-6          3
D-2          3
D-9          2
D-5          2
D-10         2
D-1, D-4     1
D-1, D-2     1
D-3          1
D-7          1
D-10, D-9    1
Name: District, dtype: int64

In [68]:
low_coverage.Agency.value_counts()

Trinity Transit                                           4
Golden Gate Bridge Highway and Transportation District    3
Yosemite Area Regional Transportation System              3
Eastern Sierra Transit Authority                          2
AC Transit                                                2
MUNI                                                      2
Redwood Coast Transit                                     2
Santa Cruz Metropolitan Transit District                  2
Mendocino Transit Authority                               2
Arcata and Mad River Transit System                       1
Blue Lake Rancheria                                       1
Capitol Corridor                                          1
Eureka Transit Service                                    1
Gold Country Stage                                        1
Humboldt Transit Authority                                1
Kern Transit                                              1
Los Angeles Department of Transportation

### Maps

In [69]:
# Grab the names of routes are "low coverage."
low_coverage_routes = low_coverage["Long Route Name"].unique().tolist()

In [70]:
# Get original geometry
low_coverage_og_geometry = (
    all_routes[all_routes["long_route_name"].isin(low_coverage_routes)]
).reset_index(drop=True)

In [71]:
att_map = att.explore(
    tiles="CartoDB positron",
    width=800,
    height=500,
)

In [72]:
att_map = low_coverage_og_geometry.explore("long_route_name", m=att_map, legend=False)

In [74]:
# att_map